1. Data Preprocessing:
Clean and preprocess your text data as needed.
Tokenize the text into words or subwords.

2. BERT Embeddings:
Use Hugging Face's Transformers library to load a pre-trained BERT model.
Obtain contextualized word embeddings for each word in your text data.

3.Topic Modeling:
Apply a topic modeling technique (e.g., LDA or NMF) to extract latent topics from the preprocessed text data.

4. Feature Extraction:
Combine the BERT embeddings and topic distributions for each document. You can concatenate or merge these features.

5. Model Training:
Choose a classification model (e.g., logistic regression, random forest, or a neural network) for each dichotomy.
Train each model on the combined feature matrix, using the corresponding labels.

6. Evaluation:
Evaluate the performance of each model on a testing set using appropriate metrics.

7. Hyperparameter Tuning:
Fine-tune the hyperparameters of each model to optimize performance.

8. Interpretation:
Analyze the importance of different features, including BERT embeddings and topic distributions, for each dichotomy to gain insights into how they contribute to personality type prediction.
By building 

In [1]:
import pandas as pd
import string
import re
import nltk
import torch
from transformers import BertModel, BertTokenizer
import sklearn
import numpy as np
import tensorflow

c:\Users\bella\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# pip install tensorflow

In [3]:
df_all_cleaned = pd.read_csv(r'C:\Users\bella\Downloads\Y3Q2 langai\df_all_cleaned.csv')

In [4]:
# df_all_cleaned = df_all_cleaned.drop(['post_feeling', 'post_judging', 'post_sensing'], axis=1)
df_all_cleaned

,auhtor_ID,post_extrovert,extrovert,post_feeling,feeling,post_judging,judging,post_sensing,sensing
0,t2_12bhu7,I wear a Lorna shore shirt out alot in public ...,1.0,I wear a Lorna shore shirt out alot in public ...,1.0,I wear a Lorna shore shirt out alot in public ...,0.0,I wear a Lorna shore shirt out alot in public ...,0.0
1,t2_12jbpd,I'd say this is a very accurate characterizati...,1.0,I'd say this is a very accurate characterizati...,0.0,I'd say this is a very accurate characterizati...,0.0,I'd say this is a very accurate characterizati...,0.0
2,t2_12uwr5,Ya know like most people with home decorations...,0.0,Ya know like most people with home decorations...,0.0,Ya know like most people with home decorations...,1.0,Ya know like most people with home decorations...,0.0
3,t2_12zm15,It's true tho. They're kinda more interesting ...,0.0,It's true tho. They're kinda more interesting ...,1.0,It's true tho. They're kinda more interesting ...,0.0,It's true tho. They're kinda more interesting ...,0.0
4,t2_13cjjl,"Yeah, but that's one of the things that make m...",0.0,"Yeah, but that's one of the things that make m...",0.0,"Yeah, but that's one of the things that make m...",0.0,"Yeah, but that's one of the things that make m...",1.0
...,...,...,...,...,...,...,...,...,...
150,t2_vfp8y,so change profession then. this would be inadm...,0.0,so change profession then. this would be inadm...,0.0,so change profession then. this would be inadm...,1.0,so change profession then. this would be inadm...,0.0
151,t2_w0842,The technological singularity. And the possibi...,0.0,The technological singularity. And the possibi...,0.0,The technological singularity. And the possibi...,1.0,The technological singularity. And the possibi...,0.0
152,t2_w6rgl,Dear God man. Chill. I'm not Einstein or Hawki...,0.0,Dear God man. Chill. I'm not Einstein or Hawki...,0.0,Dear God man. Chill. I'm not Einstein or Hawki...,1.0,Dear God man. Chill. I'm not Einstein or Hawki...,0.0
153,t2_wilcwvo,That's what a fake lib would say [Human blood ...,1.0,That's what a fake lib would say [Human blood ...,0.0,That's what a fake lib would say [Human blood ...,0.0,That's what a fake lib would say [Human blood ...,0.0


In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Join the tokens back into a single string
    processed_text = ' '.join(tokens)

    return processed_text



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bella\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bella\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
df_all_cleaned['processed_post'] = df_all_cleaned['post_extrovert'].apply(preprocess_text)
df_all_cleaned

,auhtor_ID,post_extrovert,extrovert,post_feeling,feeling,post_judging,judging,post_sensing,sensing,processed_post
0,t2_12bhu7,I wear a Lorna shore shirt out alot in public ...,1.0,I wear a Lorna shore shirt out alot in public ...,1.0,I wear a Lorna shore shirt out alot in public ...,0.0,I wear a Lorna shore shirt out alot in public ...,0.0,wear lorna shore shirt alot public lewd long s...
1,t2_12jbpd,I'd say this is a very accurate characterizati...,1.0,I'd say this is a very accurate characterizati...,0.0,I'd say this is a very accurate characterizati...,0.0,I'd say this is a very accurate characterizati...,0.0,id say accurate characterization ni users read...
2,t2_12uwr5,Ya know like most people with home decorations...,0.0,Ya know like most people with home decorations...,0.0,Ya know like most people with home decorations...,1.0,Ya know like most people with home decorations...,0.0,ya know like people home decorations could sav...
3,t2_12zm15,It's true tho. They're kinda more interesting ...,0.0,It's true tho. They're kinda more interesting ...,1.0,It's true tho. They're kinda more interesting ...,0.0,It's true tho. They're kinda more interesting ...,0.0,true tho theyre kinda interesting buuuut issue...
4,t2_13cjjl,"Yeah, but that's one of the things that make m...",0.0,"Yeah, but that's one of the things that make m...",0.0,"Yeah, but that's one of the things that make m...",0.0,"Yeah, but that's one of the things that make m...",1.0,yeah thats one things make better objectively ...
...,...,...,...,...,...,...,...,...,...,...
150,t2_vfp8y,so change profession then. this would be inadm...,0.0,so change profession then. this would be inadm...,0.0,so change profession then. this would be inadm...,1.0,so change profession then. this would be inadm...,0.0,change profession would inadmissible country p...
151,t2_w0842,The technological singularity. And the possibi...,0.0,The technological singularity. And the possibi...,0.0,The technological singularity. And the possibi...,1.0,The technological singularity. And the possibi...,0.0,technological singularity possibility contribu...
152,t2_w6rgl,Dear God man. Chill. I'm not Einstein or Hawki...,0.0,Dear God man. Chill. I'm not Einstein or Hawki...,0.0,Dear God man. Chill. I'm not Einstein or Hawki...,1.0,Dear God man. Chill. I'm not Einstein or Hawki...,0.0,dear god man chill im einstein hawking serious...
153,t2_wilcwvo,That's what a fake lib would say [Human blood ...,1.0,That's what a fake lib would say [Human blood ...,0.0,That's what a fake lib would say [Human blood ...,0.0,That's what a fake lib would say [Human blood ...,0.0,thats fake lib would say human blood water url...


In [7]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Function to obtain BERT embeddings for a text
def get_bert_embeddings(text):
    # Tokenize input text and convert to tensor
    tokens = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt', max_length=512, truncation=True)

    # Get BERT embeddings
    with torch.no_grad():
        outputs = bert_model(tokens)
        embeddings = outputs.last_hidden_state

    # Average the embeddings across tokens (you can modify this based on your needs)
    avg_embedding = torch.mean(embeddings, dim=1).squeeze().numpy()

    return avg_embedding

# # Example DataFrame with a 'processed_post' column
# data = {'processed_post': ["enjoy hiking spending time nature",
#                             "text data preprocessing crucial nlp tasks",
#                             "stop words removal improves text analysis"]}
# df = pd.DataFrame(data)

# # Apply BERT embeddings to the 'processed_post' column
# df['bert_embeddings'] = df['processed_post'].apply(get_bert_embeddings)

# # Display the DataFrame with processed text and BERT embeddings
# print(df)

df_all_cleaned['bert_embeddings'] = df_all_cleaned['post_extrovert'].apply(get_bert_embeddings)
print(df_all_cleaned)



      auhtor_ID                                     post_extrovert  extrovert  \
0     t2_12bhu7  I wear a Lorna shore shirt out alot in public ...        1.0   
1     t2_12jbpd  I'd say this is a very accurate characterizati...        1.0   
2     t2_12uwr5  Ya know like most people with home decorations...        0.0   
3     t2_12zm15  It's true tho. They're kinda more interesting ...        0.0   
4     t2_13cjjl  Yeah, but that's one of the things that make m...        0.0   
..          ...                                                ...        ...   
150    t2_vfp8y  so change profession then. this would be inadm...        0.0   
151    t2_w0842  The technological singularity. And the possibi...        0.0   
152    t2_w6rgl  Dear God man. Chill. I'm not Einstein or Hawki...        0.0   
153  t2_wilcwvo  That's what a fake lib would say [Human blood ...        1.0   
154   t2_zq7gkv  My biggest problem is asking for it. I don’t n...        1.0   

                           

In [8]:
df_all_cleaned

,auhtor_ID,post_extrovert,extrovert,post_feeling,feeling,post_judging,judging,post_sensing,sensing,processed_post,bert_embeddings
0,t2_12bhu7,I wear a Lorna shore shirt out alot in public ...,1.0,I wear a Lorna shore shirt out alot in public ...,1.0,I wear a Lorna shore shirt out alot in public ...,0.0,I wear a Lorna shore shirt out alot in public ...,0.0,wear lorna shore shirt alot public lewd long s...,"[0.03740044, 0.03744348, 0.40402788, -0.154586..."
1,t2_12jbpd,I'd say this is a very accurate characterizati...,1.0,I'd say this is a very accurate characterizati...,0.0,I'd say this is a very accurate characterizati...,0.0,I'd say this is a very accurate characterizati...,0.0,id say accurate characterization ni users read...,"[-0.122634806, 0.06978733, 0.23516777, -0.1674..."
2,t2_12uwr5,Ya know like most people with home decorations...,0.0,Ya know like most people with home decorations...,0.0,Ya know like most people with home decorations...,1.0,Ya know like most people with home decorations...,0.0,ya know like people home decorations could sav...,"[0.10358047, -0.079817355, 0.4862674, 0.006832..."
3,t2_12zm15,It's true tho. They're kinda more interesting ...,0.0,It's true tho. They're kinda more interesting ...,1.0,It's true tho. They're kinda more interesting ...,0.0,It's true tho. They're kinda more interesting ...,0.0,true tho theyre kinda interesting buuuut issue...,"[-0.11131706, 0.070213296, 0.5168624, 0.017204..."
4,t2_13cjjl,"Yeah, but that's one of the things that make m...",0.0,"Yeah, but that's one of the things that make m...",0.0,"Yeah, but that's one of the things that make m...",0.0,"Yeah, but that's one of the things that make m...",1.0,yeah thats one things make better objectively ...,"[0.21926472, 0.11031033, 0.28619415, 0.1073027..."
...,...,...,...,...,...,...,...,...,...,...,...
150,t2_vfp8y,so change profession then. this would be inadm...,0.0,so change profession then. this would be inadm...,0.0,so change profession then. this would be inadm...,1.0,so change profession then. this would be inadm...,0.0,change profession would inadmissible country p...,"[-0.17524596, 0.1841626, 0.44777465, -0.118973..."
151,t2_w0842,The technological singularity. And the possibi...,0.0,The technological singularity. And the possibi...,0.0,The technological singularity. And the possibi...,1.0,The technological singularity. And the possibi...,0.0,technological singularity possibility contribu...,"[-0.025105778, -0.08084041, 0.3463775, -0.0189..."
152,t2_w6rgl,Dear God man. Chill. I'm not Einstein or Hawki...,0.0,Dear God man. Chill. I'm not Einstein or Hawki...,0.0,Dear God man. Chill. I'm not Einstein or Hawki...,1.0,Dear God man. Chill. I'm not Einstein or Hawki...,0.0,dear god man chill im einstein hawking serious...,"[0.088415004, 0.22571912, 0.38455746, -0.03194..."
153,t2_wilcwvo,That's what a fake lib would say [Human blood ...,1.0,That's what a fake lib would say [Human blood ...,0.0,That's what a fake lib would say [Human blood ...,0.0,That's what a fake lib would say [Human blood ...,0.0,thats fake lib would say human blood water url...,"[0.05957357, 0.10176626, 0.3839616, -0.0742901..."


In [11]:
# from sklearn.decomposition import LatentDirichletAllocation
# from sklearn.feature_extraction.text import CountVectorizer

# # Function to apply LDA and return topic distributions
# def get_topic_distribution(text):
#     vectorizer = CountVectorizer(max_features=1000, stop_words='english')
#     text_vectorized = vectorizer.fit_transform([text])
    
#     lda = LatentDirichletAllocation(n_components=5, random_state=42)  # Adjust the number of topics as needed
#     topic_distribution = lda.fit_transform(text_vectorized)
    
#     return topic_distribution.flatten()

# # Assume 'processed_post' is the column with processed text
# df_all_cleaned['topic_distribution'] = df_all_cleaned['processed_post'].apply(get_topic_distribution)

# # Display the DataFrame with combined features
# print(df_all_cleaned)

In [12]:
# # Combine BERT embeddings and topic distributions
# # df_all_cleaned['combined_features'] = df_all_cleaned.apply(lambda row: np.concatenate([row['bert_embeddings'], row['topic_distribution']]), axis=1)

# df_all_cleaned['combined_features'] = df_all_cleaned.apply(lambda row: 0.5 * (row['bert_embeddings'] + row['topic_distribution']), axis=1)


# # Display the DataFrame with combined features
# print(df_all_cleaned)


In [13]:
df_all_cleaned

,auhtor_ID,post_extrovert,extrovert,post_feeling,feeling,post_judging,judging,post_sensing,sensing,processed_post,bert_embeddings,topic_distribution
0,t2_12bhu7,I wear a Lorna shore shirt out alot in public ...,1.0,I wear a Lorna shore shirt out alot in public ...,1.0,I wear a Lorna shore shirt out alot in public ...,0.0,I wear a Lorna shore shirt out alot in public ...,0.0,wear lorna shore shirt alot public lewd long s...,"[0.03740044, 0.03744348, 0.40402788, -0.154586...","[0.9997959152018512, 5.102119953720001e-05, 5...."
1,t2_12jbpd,I'd say this is a very accurate characterizati...,1.0,I'd say this is a very accurate characterizati...,0.0,I'd say this is a very accurate characterizati...,0.0,I'd say this is a very accurate characterizati...,0.0,id say accurate characterization ni users read...,"[-0.122634806, 0.06978733, 0.23516777, -0.1674...","[1.941758505806631e-05, 1.9417585058066305e-05..."
2,t2_12uwr5,Ya know like most people with home decorations...,0.0,Ya know like most people with home decorations...,0.0,Ya know like most people with home decorations...,1.0,Ya know like most people with home decorations...,0.0,ya know like people home decorations could sav...,"[0.10358047, -0.079817355, 0.4862674, 0.006832...","[0.9999582903714797, 1.0427407129993352e-05, 1..."
3,t2_12zm15,It's true tho. They're kinda more interesting ...,0.0,It's true tho. They're kinda more interesting ...,1.0,It's true tho. They're kinda more interesting ...,0.0,It's true tho. They're kinda more interesting ...,0.0,true tho theyre kinda interesting buuuut issue...,"[-0.11131706, 0.070213296, 0.5168624, 0.017204...","[0.9999588540280505, 1.0286492987415484e-05, 1..."
4,t2_13cjjl,"Yeah, but that's one of the things that make m...",0.0,"Yeah, but that's one of the things that make m...",0.0,"Yeah, but that's one of the things that make m...",0.0,"Yeah, but that's one of the things that make m...",1.0,yeah thats one things make better objectively ...,"[0.21926472, 0.11031033, 0.28619415, 0.1073027...","[0.9997812483957159, 5.4687901070990636e-05, 5..."
...,...,...,...,...,...,...,...,...,...,...,...,...
150,t2_vfp8y,so change profession then. this would be inadm...,0.0,so change profession then. this would be inadm...,0.0,so change profession then. this would be inadm...,1.0,so change profession then. this would be inadm...,0.0,change profession would inadmissible country p...,"[-0.17524596, 0.1841626, 0.44777465, -0.118973...","[1.265631868878355e-05, 1.2656318688783549e-05..."
151,t2_w0842,The technological singularity. And the possibi...,0.0,The technological singularity. And the possibi...,0.0,The technological singularity. And the possibi...,1.0,The technological singularity. And the possibi...,0.0,technological singularity possibility contribu...,"[-0.025105778, -0.08084041, 0.3463775, -0.0189...","[2.957185171821599e-05, 2.957185171821599e-05,..."
152,t2_w6rgl,Dear God man. Chill. I'm not Einstein or Hawki...,0.0,Dear God man. Chill. I'm not Einstein or Hawki...,0.0,Dear God man. Chill. I'm not Einstein or Hawki...,1.0,Dear God man. Chill. I'm not Einstein or Hawki...,0.0,dear god man chill im einstein hawking serious...,"[0.088415004, 0.22571912, 0.38455746, -0.03194...","[0.9999604284773709, 9.892880657306835e-06, 9...."
153,t2_wilcwvo,That's what a fake lib would say [Human blood ...,1.0,That's what a fake lib would say [Human blood ...,0.0,That's what a fake lib would say [Human blood ...,0.0,That's what a fake lib would say [Human blood ...,0.0,thats fake lib would say human blood water url...,"[0.05957357, 0.10176626, 0.3839616, -0.0742901...","[0.9998581033172271, 3.547417069318599e-05, 3...."


In [36]:
%%time

df_labels =  pd.DataFrame()
df_labels['extrovert']= df_all_cleaned['extrovert']
df_labels['feeling']= df_all_cleaned['feeling']
df_labels['judging']= df_all_cleaned['judging']
df_labels['sensing']= df_all_cleaned['sensing']

df_labels

CPU times: total: 0 ns
Wall time: 1.99 ms


,extrovert,feeling,judging,sensing
0,1.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0
...,...,...,...,...
150,0.0,0.0,1.0,0.0
151,0.0,0.0,1.0,0.0
152,0.0,0.0,1.0,0.0
153,1.0,0.0,0.0,0.0


In [37]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, df_labels, test_size=0.2, random_state=42)

# Build a simple neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(df_labels.shape[1], activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Predictions
nn_predictions = model.predict(X_test)

Epoch 1/10
4/4 [==============================] - 1s 58ms/step - loss: 0.6595 - accuracy: 0.2581 - val_loss: 0.5479 - val_accuracy: 0.3226
Epoch 2/10
4/4 [==============================] - 0s 10ms/step - loss: 0.5527 - accuracy: 0.3710 - val_loss: 0.5740 - val_accuracy: 0.3226
Epoch 3/10
4/4 [==============================] - 0s 11ms/step - loss: 0.5540 - accuracy: 0.3710 - val_loss: 0.5774 - val_accuracy: 0.3226
Epoch 4/10
4/4 [==============================] - 0s 10ms/step - loss: 0.5451 - accuracy: 0.3710 - val_loss: 0.5615 - val_accuracy: 0.3226
Epoch 5/10
4/4 [==============================] - 0s 10ms/step - loss: 0.5394 - accuracy: 0.3710 - val_loss: 0.5435 - val_accuracy: 0.3226
Epoch 6/10
4/4 [==============================] - 0s 12ms/step - loss: 0.5290 - accuracy: 0.3710 - val_loss: 0.5434 - val_accuracy: 0.3226
Epoch 7/10
4/4 [==============================] - 0s 12ms/step - loss: 0.5223 - accuracy: 0.3710 - val_loss: 0.5474 - val_accuracy: 0.3226
Epoch 8/10
4/4 [===========

In [38]:
# Evaluate for each df_lables
for column, true_labels in df_labels.iteritems():
    i = df_labels.columns.get_loc(column)  # Get the index of the current column
    threshold = 0.5  # Adjust the threshold based on your task
    binary_predictions = (nn_predictions[:, i] > threshold).astype(int)
    accuracy = accuracy_score(y_test[column], binary_predictions)
    print(f"Accuracy for {column} (Neural Network): {accuracy}")

    # Classification report for each dichotomy
    print(f"Classification Report for {column} (Neural Network):")
    print(classification_report(y_test[column], binary_predictions))

Accuracy for extrovert (Neural Network): 0.8064516129032258
Classification Report for extrovert (Neural Network):
              precision    recall  f1-score   support

         0.0       0.81      1.00      0.89        25
         1.0       0.00      0.00      0.00         6

    accuracy                           0.81        31
   macro avg       0.40      0.50      0.45        31
weighted avg       0.65      0.81      0.72        31

Accuracy for feeling (Neural Network): 0.7419354838709677
Classification Report for feeling (Neural Network):
              precision    recall  f1-score   support

         0.0       0.74      1.00      0.85        23
         1.0       0.00      0.00      0.00         8

    accuracy                           0.74        31
   macro avg       0.37      0.50      0.43        31
weighted avg       0.55      0.74      0.63        31

Accuracy for judging (Neural Network): 0.5161290322580645
Classification Report for judging (Neural Network):
            

C:\Users\bella\AppData\Local\Temp\ipykernel_6576\1734260993.py:2: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, true_labels in df_labels.iteritems():
c:\Users\bella\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bella\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bella\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_cl

In [ ]:
# tokenization 

def tokenize_documents(df):
    tokenized_documents = []

    for post in df['post_extrovert']:
        token_list = []

        for token in post.split(' '):
            if token[-1] in string.punctuation:
                token_list += [token[:-1], token[-1]]
            else:
                token_list.append(token)

        tokenized_documents.append(token_list)

    df['tokenized_post'] = tokenized_documents
    return df

In [ ]:
df_all_cleaned = tokenize_documents(df_all_cleaned)
df_all_cleaned

In [ ]:
# vectorization

from collections import Counter

def vectorize_documents(df):
    M = []
    T = sorted(set([token for tokens in df['tokenized_post'] for token in tokens]))

    for tokens in df['tokenized_post']:
        vector = []
        freq_dict = Counter(tokens)
        
        for t in T:
            vector.append(freq_dict[t])

        M.append(vector)

    df['document_term_matrix'] = M
    return df, T

In [ ]:
df_all_cleaned, terms = vectorize_documents(df_all_cleaned)
df_all_cleaned

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# # Assuming df_all_cleaned is your DataFrame with 'tokenized_post' column
# documents = [' '.join(tokens) for tokens in df_all_cleaned['tokenized_post']]

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the documents
tfidf_matrix = tfidf_vectorizer.fit_transform(df_all_cleaned['post_extrovert'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Concatenate the TF-IDF DataFrame with the original DataFrame
df_all_cleaned = pd.concat([df_all_cleaned, tfidf_df], axis=1)


In [ ]:
df_all_cleaned